# Access GitHub via OpenAPI

In this notebook we shall attempt to access GitHub via their OpenAPI interface.

Steps:
1. Fetch GitHub OpenAPI specification
2. Examine the specification
3. Generate a Julia client package using that
4. Examine the generated code
5. Examine the generated docs
6. Use it!

In [1]:
# Clean up before starting
rm("GitHubClient"; force=true, recursive=true)
# Add required packages
# Pkg.add(["OpenAPI", "JSON"])

In [2]:
using Downloads, OpenAPI, JSON

## Fetch the specification

GitHub makes available their OpenAPI specification at:
https://github.com/github/rest-api-description

In [3]:
spec_file = "specifications/api.github.com.json" 
spec_source = "https://raw.githubusercontent.com/github/rest-api-description/main/descriptions/api.github.com/api.github.com.json"

if !isfile(spec_file)
    Downloads.download(spec_source, spec_file)
end

## Examine the specification

We can examine the API specifications
- Either by opening the JSON specification directly
- Or by opening it in some tool, e.g. the Swagger UI

The Swagger UI tool can be invoked using the OpenAPI package. It uses docker to launch the Swagger tools container, which provides the UI.

In [4]:
# Examine API specifications
OpenAPI.swagger_ui("specifications/api.github.com.json")

29278e132ad4afa9f4e4e18408aaaf0f49e319cde02858d78ba55b8bf2fadeeb


"http://localhost:8080"

In [5]:
# Stop the UI
OpenAPI.stop_swagger();

## Code generation

Use openapi-generator to generate the Julia package.
It is available bundled in a convenient docker image,
which lets us use it without having to install anything.

In [6]:
# Start the codegen service
OpenAPI.openapi_generator()

a1ee91f5b289d658fed6f1db91484c2181524e05ea6db9831eac354506b7c358


"http://localhost:8080"

In [7]:
spec = JSON.parsefile("specifications/api.github.com.json")
OpenAPI.generate(spec;
    type=:client,
    package_name="GitHubClient",
    output_dir="GitHubClient",
);


7-Zip (a) [64] 17.04 : Copyright (c) 1999-2021 Igor Pavlov : 2017-08-28
p7zip Version 17.04 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,12 CPUs Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz (906EA),ASM,AES-NI)

Scanning the drive for archives:
1 file, 2775236 bytes (2711 KiB)

Extracting archive: /tmp/jl_OPqacT/generated.zip
--
Path = /tmp/jl_OPqacT/generated.zip
Type = zip
Physical Size = 2775236

Everything is Ok

Files: 2956
Size:       11302300
Compressed: 2775236


In [8]:
# Stop the codegen service
OpenAPI.stop_openapi_generator();

true

## Examine the generated code

The code is generated as a Julia module.
The module can be embedded into another package.

Or it can also be registered directly as a Julia package on its own. Just need to add Project.toml.

README.md contains docs for all APIs.

In [9]:
run(`ls -la GitHubClient`);

total 392
drwxrwxr-x 5 tan tan   4096 Jul  6 09:37 .
drwxrwxr-x 5 tan tan   4096 Jul  6 09:38 ..
drwxrwxr-x 2 tan tan  90112 Jul  6 09:37 docs
drwxrwxr-x 2 tan tan   4096 Jul  6 09:37 .openapi-generator
-rw-rw-r-- 1 tan tan   1040 Jul  6 09:37 .openapi-generator-ignore
-rw-rw-r-- 1 tan tan 283195 Jul  6 09:37 README.md
drwxrwxr-x 4 tan tan   4096 Jul  6 09:37 src


In [10]:
run(`ls -la GitHubClient/src`);

total 204
drwxrwxr-x 4 tan tan   4096 Jul  6 09:37 .
drwxrwxr-x 5 tan tan   4096 Jul  6 09:37 ..
drwxrwxr-x 2 tan tan   4096 Jul  6 09:37 apis
-rw-rw-r-- 1 tan tan   1462 Jul  6 09:37 GitHubClient.jl
-rw-rw-r-- 1 tan tan  81211 Jul  6 09:37 modelincludes.jl
drwxrwxr-x 2 tan tan 106496 Jul  6 09:37 models


## Use Generated Package to Invoke APIs

The APIs can be invoked just like Julia methods.
All the HTTP/REST communication and serialization and deserialization are handled transparently.

In [11]:
include("GitHubClient/src/GitHubClient.jl")
using .GitHubClient

In [12]:
# Create a client for the API group which has the API of interest
client = OpenAPI.Clients.Client(GitHubClient.basepath(GitHubClient.ReposApi));
reposapi = GitHubClient.ReposApi(client);

In [13]:
# Invoke APIs as regular Julia methods
# API responds with the results converted to a Julia type
# It also returns the raw HTTP response as the second return value, for advanced use
julia_repo, _api_resp = GitHubClient.repos_get(reposapi, "julialang", "julia")

julia_repo

{
  "pulls_url": "https://api.github.com/repos/JuliaLang/julia/pulls{/number}",
  "organization": {
    "repos_url": "https://api.github.com/users/JuliaLang/repos",
    "login": "JuliaLang",
    "gists_url": "https://api.github.com/users/JuliaLang/gists{/gist_id}",
    "site_admin": false,
    "subscriptions_url": "https://api.github.com/users/JuliaLang/subscriptions",
    "id": 743164,
    "html_url": "https://github.com/JuliaLang",
    "node_id": "MDEyOk9yZ2FuaXphdGlvbjc0MzE2NA==",
    "followers_url": "https://api.github.com/users/JuliaLang/followers",
    "organizations_url": "https://api.github.com/users/JuliaLang/orgs",
    "starred_url": "https://api.github.com/users/JuliaLang/starred{/owner}{/repo}",
    "avatar_url": "https://avatars.githubusercontent.com/u/743164?v=4",
    "events_url": "https://api.github.com/users/JuliaLang/events{/privacy}",
    "url": "https://api.github.com/users/JuliaLang",
    "received_events_url": "https://api.github.com/users/JuliaLang/received_even

In [14]:
# The return value is made available as a Julia struct

println("The ", julia_repo.name, " repo has ", julia_repo.stargazers_count, " stars!")

The julia repo has 44965 stars!


In [15]:
# Clean up before closing
rm("GitHubClient"; force=true, recursive=true)